In [7]:
from parlai.core.agents import create_agent
from parlai.agents.emely.emely import EmelyAgent
from parlai.core.opt import Opt
from pathlib import Path
import torch
from parlai.scripts.torchscript import export_emely
from parlai.core.agents import create_agent

# Initialize model settings

model_path = Path.cwd() / 'data/models/blender/blender_90M/'
assert model_path.is_dir()

opt_path = model_path / 'model.opt'
opt = Opt.load(opt_path)

# Change opts
opt['skip_generation'] = False
opt['init_model'] = (model_path / 'model').as_posix()
opt['no_cuda'] = True  # Cloud run doesn't offer gpu support

# Inference options
opt['inference'] = 'greedy' # 'beam'
opt['beam_size'] = 1

opt["scripted-model-file"] = "emely_scripted_test.pt"
opt["script-module"] = "parlai.torchscript.modules:TorchScriptGreedySearch"
opt["model_file"] = opt["init_model"]

opt["temp_separator"] = "__space__"

opt["bpe_add_prefix_space"] = False

In [2]:
# Test exporting to torchscript
export_emely(opt)

22:55:56 | loading dictionary from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model.dict
22:55:56 | num words = 54944
22:55:56 | DEPRECATED: XLM should only be used for backwards compatibility, as it involves a less-stable layernorm operation.
22:55:57 | Total parameters: 87,508,992 (87,508,992 trainable)
22:55:57 | Loading existing model params from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model


/home/ckjellson/code/emely-models/ParlAI/parlai/agents/transformer/modules/encoder.py:183: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if positions.max().item() > self.n_positions:
/home/ckjellson/code/emely-models/ParlAI/parlai/agents/transformer/modules/attention.py:147: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert (
/home/ckjellson/anaconda3/envs/emelymodels/lib/python3.8/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'tr

torch.Size([1, 3])
2
{'0__self_attn__prev_key': tensor([[[[ 1.7526, -0.5386, -1.2811,  ..., -1.9779, -1.0670,  0.1217],
          [ 0.9178, -1.5451,  1.6102,  ..., -1.7043,  1.3127,  0.0156]],

         [[ 0.2943,  2.2358,  3.5683,  ..., -3.8988,  0.5640,  1.3951],
          [ 0.7878, -0.2087,  1.0752,  ..., -0.7248, -3.9533, -1.2152]],

         [[ 0.0442,  0.0730,  0.2132,  ..., -0.1067, -1.1602,  0.5259],
          [-0.1513,  0.5204, -0.8468,  ..., -0.7351, -0.4042, -1.2901]],

         ...,

         [[-0.2849,  0.3234, -1.1044,  ...,  0.6337,  2.4490, -0.5633],
          [-1.4803,  0.4747, -3.3109,  ...,  0.9261,  0.5223,  1.6246]],

         [[ 0.7337, -0.6476,  1.1465,  ...,  2.0486, -0.6465,  1.4505],
          [-0.0802, -1.5693,  1.0851,  ..., -0.1458, -0.6731, -0.1392]],

         [[ 1.1624, -0.3463,  0.1366,  ...,  0.7243,  0.9684, -1.8976],
          [-1.4605, -0.4311, -0.1438,  ..., -1.7887,  1.4845,  0.3190]]]],
       grad_fn=<IndexSelectBackward>), '0__self_attn__prev_v

RuntimeError: torch.cat(): Sizes of tensors must match except in dimension 2. Got 2 and 1 in dimension 1 (The offending index is 1)

In [8]:
# Create model to test tokenization etc
emely_agent = EmelyAgent(opt)

08:12:05 | loading dictionary from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model.dict
08:12:05 | num words = 54944
08:12:06 | Total parameters: 87,508,992 (87,508,992 trainable)
08:12:06 | Loading existing model params from /home/ckjellson/code/emely-models/ParlAI/data/models/blender/blender_90M/model


In [9]:
from parlai.utils.bpe import SubwordBPEHelper
from parlai.torchscript.modules_emely import ScriptableSubwordBpeHelper
sbpe = SubwordBPEHelper(emely_agent.opt)
joint_bpe_codes = {}
for k in sbpe.bpe.bpe_codes.keys():
    joint_bpe_codes[emely_agent.opt["temp_separator"].join(k)] = sbpe.bpe.bpe_codes[k]
ssbpe = ScriptableSubwordBpeHelper(True, joint_bpe_codes, sbpe.bpe.separator, emely_agent.opt["temp_separator"])
o1 = sbpe.encode("Hi! What is your name?\nI'm Christoffer.")
o2 = ssbpe.helper_encode("Hi! What is your name?\nI'm Christoffer.")
print(o1)
print(o2)

['H@@', 'i', '!', 'W@@', 'hat', 'is', 'your', 'name', '?', '__newln__', 'I', "'", 'm', 'C@@', 'h@@', 'ri@@', 'st@@', 'offer', '.']
['H@@', 'i', '!', 'W@@', 'hat', 'is', 'your', 'name', '?', '__newln__', 'I', "'", 'm', 'C@@', 'h@@', 'ri@@', 'st@@', 'offer', '.']
